# Problem 4. Independent components analysis

While studying Independent Component Analysis (ICA) in class, we made an informal argument about why Gaussian distributed sources will not work. We also mentioned that any other distribution (except Gaussian) for the sources will work for ICA, and hence used the logistic distribution instead. In this problem, we will go deeper into understanding why Gaussian distributed sources are a problem. We will also derive ICA with the Laplace distribution, and apply it to the cocktail party problem.

Reintroducing notation, let $s\in\mathbb{R}^n$ be source data that is generated from $n$ independent sources. Let $x\in\mathbb{R}^n$ be observed data such that $x=As$, where $A\in\mathbb{R}^{n\times n}$ is called the mixing matrix. We assume $A$ is invertible, and $W=A^{-1}$ is called the *unmixing matrix*. So, $s=Wx$. The goal of ICA is to estimate $W$. Similar to the notes, we denote $w_j^T$ to be the $j^{th}$ row of $W$. Note that this implies that the $j^{th}$ source can be reconstructed with $w_j$ and $x$, since $s_j=w_j^tx$. We are given a training set $\{x^{(1)},\ldots,x^{(m)}\}$ for the following sub-questions. Let us denote the entire training set by the design matrix $X\in\mathbb{R}^{m\times n}$ where each example corresponds to a row in the matrix.

## (a) [5 points] Gaussian source
For this sub-question, we assume sources are distributed according to a standard normal
distribution, i.e $s_j\sim \mathcal{N}(0,1), j=1,\ldots,m$. The likelihood of our unmixing matrix, as described in the notes, is

\begin{align*}
\ell(W) = \sum_{i=1}^m\left(\log |W| + \sum_{j=1}^n\log g'(w^T_jx^{(i)})\right),
\end{align*}

where $g$ is the cumulative distribution function, and $g'$ is the probability density function of the source distribution (in this sub-question it is a standard normal distribution). Whereas in the notes we derive an update rule to train $W$ iteratively, for the cause of Gaussian distributed sources, we can analytically reason about the resulting $W$.
Try to derive a closed form expression for W in terms of $X$ when $g$ is the standard normal CDF. Deduce the relation between $W$ and $X$ in the simplest terms, and highlight the ambiguity (in terms of rotational invariance) in computing $W$.

### Answer:

Plugging 
$g'(w^T_jx^{(i)}) = \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{1}{2}w^T_jx^{(i)}{x^{(i)}}^Tw_j\right)$ 
into the formula of $\ell(W)$, we obtain 

\begin{align*}
\ell(W) 
& = C + \sum_{i=1}^m\left(\log |W| + \sum_{j=1}^n   -\frac{1}{2}w^T_jx^{(i)}{x^{(i)}}^Tw_j\right)\\
& = C + m \log |W| -\frac{1}{2}\sum_{i=1}^m\sum_{j=1}^n   w^T_jx^{(i)}{x^{(i)}}^Tw_j\\
& = C + m \log |W| -\frac{1}{2}\sum_{i=1}^m   Wx^{(i)}(Wx^{(i)})^T\\
& = C + m \log |W| -\frac{1}{2}tr\left(WX(WX)^T\right)\\
& = C + m \log |W| -\frac{1}{2}tr\left(WXX^TW^T\right).
\end{align*}

Computing and setting $\nabla_W(\ell)$ to zero, we have

\begin{align*}
\nabla_W(\ell) 
& = m (W^T)^{-1} -WXX^T\\
& = 0,
\end{align*}

which implies 

\begin{align*}
W^TW = m(XX^T)^{-1}.
\end{align*}

Therefore, any $W$ maximizing $\ell$ satisfies this equality. Let $W$ be such a matrix, $R\in \mathbb{R}^{n\times n}$ an arbitrary orthonormal matrix, and set $W'= RW$. In this situation,

\begin{align*}
W'^TW'
& = W^TR^TRW\\
& = W^TW.
\end{align*}

Using the fact that $|W'|=|W|$, we obtain 
\begin{align*}
\ell(W') 
& = C + m \log |W'| -\frac{1}{2}tr\left(W'XX^TW'^T\right)\\
& = C + m \log |W| -\frac{1}{2}tr\left(W'^TW'XX^T\right)\\
& = C + m \log |W| -\frac{1}{2}tr\left(W^TWXX^T\right)\\
& = C + m \log |W| -\frac{1}{2}tr\left(WXX^TW^T\right)\\
& = \ell(W)
\end{align*}
which means that $W'$ maximizes $\ell$ as well.
Therefore, there is no way to say if $W$ should be use as unmixing matrix or $W'$.

## (b) [10 points] Laplace source.
For this sub-question, we assume sources are distributed according to a standard Laplace
distribution, i.e $s_i\sim \mathcal{L}(0,1)$. The Laplace distribution $\mathcal{L}(0,1)$ has PDF $\mathcal{f}_{\mathcal{L}}(s)=\frac{1}{2}\exp(-|s|)$. With this assumption, derive the update rule for a single example in the form

\begin{align*}
W := W +\alpha(\cdots).
\end{align*}

### Answer:

We plug $g'(s) = \frac{1}{2}\exp(-|s|)$ into the $\ell$ sunction to obtain 
\begin{align*}
\ell(W) 
& = \sum_{i=1}^m\left(\log |W| + \sum_{j=1}^n\log g'(w^T_jx^{(i)})\right)\\
& = C + \sum_{i=1}^m\left(\log |W| - \sum_{j=1}^n |w^T_jx^{(i)}|\right).
\end{align*}

Therefore, for a training example $x^{(i)}$, 
\begin{align*}
\nabla_W\ell(W) 
& = (W^T)^{-1} - {\rm sgn}(Wx^{(i)})  {x^{(i)}}^T
\end{align*}

and the update rule for a single example is 

\begin{align*}
W := W +\alpha\left((W^T)^{-1} - {\rm sgn}(Wx^{(i)})  {x^{(i)}}^T\right).
\end{align*}

## (c) [5 points] Cocktail Party Problem
For this question you will implement the Bell and Sejnowski ICA algorithm, but assuming a Laplace source (as derived in part-b), instead of the Logistic distribution covered in class. The file `mix.dat` contains the input data which consists of a matrix with $5$ columns, with each column corresponding to one of the mixed signals $x_i$. The code for this question can be found in `p4_ica.py`.

Implement the update W and unmix functions in `p4_ica.py`.

You can then run `p4_ica.py` in order to split the mixed audio into its components. The mixed audio tracks are written to `midex_i.wav` in the output folder. The split audio tracks are written to `split_i.wav` in the output folder.

To make sure your code is correct, you should listen to the resulting unmixed sources. (Some overlap or noise in the sources may be present, but the different sources should be pretty clearly separated.)

__Note:__ In our implementation, we anneal the learning rate $\alpha$ (slowly decreased it over time) to speed up learning. In addition to using the variable learning rate to speed up convergence, one thing that we also do is choose a random permutation of the training data, and running stochastic gradient ascent visiting the training data in that order (each of the specified learning rates was then used for one full pass through the data).

In [1]:
import numpy as np
import scipy.io.wavfile

In [2]:
def update_W(W, x, learning_rate):
    """
    Perform a gradient ascent update on W using data element x and the provided learning rate.

    This function should return the updated W.

    Use the laplace distribiution in this problem.

    Args:
        W: The W matrix for ICA
        x: A single data element
        learning_rate: The learning rate to use

    Returns:
        The updated W
    """
    
    # *** START CODE HERE ***
    updated_W = W + learning_rate * (np.linalg.inv(W.T) - np.sign(W@x.reshape(-1,1))@x.reshape(1,-1))
    # *** END CODE HERE ***

    return updated_W


def unmix(X, W):
    """
    Unmix an X matrix according to W using ICA.

    Args:
        X: The data matrix
        W: The W for ICA

    Returns:
        A numpy array S containing the split data
    """

    S = np.zeros(X.shape)


    # *** START CODE HERE ***
    S =  X @ W.T 
    # *** END CODE HERE ***

    return S


Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def save_sound(audio, name):
    scipy.io.wavfile.write('output/{}.wav'.format(name), Fs, audio)

def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1 , 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01 , 0.01, 0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for lr in anneal:
        print(lr)
        rand = np.random.permutation(range(M))
        for i in rand:
            x = X[i]
            W = update_W(W, x, lr)

    return W

def main():
    X = normalize(load_data())

    print('X.shape: ', X.shape)

    for i in range(X.shape[1]):
        save_sound(X[:, i], 'mixed_{}'.format(i))

    W = unmixer(X)
    print('unmixer: ', W)
    S = normalize(unmix(X, W))

    for i in range(S.shape[1]):
        save_sound(S[:, i], 'split_{}'.format(i))

if __name__ == '__main__':
    main()



X.shape:  (53442, 5)
Separating tracks ...
0.1
0.1
0.1
0.05
0.05
0.05
0.02
0.02
0.01
0.01
0.005
0.005
0.002
0.002
0.001
0.001
unmixer:  [[ 52.83441795  16.79422341  19.94579171 -10.20014236 -20.89404044]
 [ -9.90390401  -0.97551781  -4.65143455   8.03957138   1.77059925]
 [  8.29461991  -7.47270509  19.31979841  15.18218038 -14.34599704]
 [-14.6653108  -26.64475534   2.44392828  21.37767282  -8.41925368]
 [ -0.27381437  18.38342182   9.31885365   9.10269483  30.60517381]]
